# Collecte, Traitement, et Analyse de données de réseaux sociaux

Importation des librairies

In [5]:
import json
import re
import pandas as pd
from textblob import TextBlob

Ouverture du fichier Json et nettoyage du text

In [6]:
#Ouverture du fichier jason
with open('versailles_tweets_100.json') as json_data:
    data_dict = json.load(json_data)

#Effacer tous les caractères spéciaux dans le texte

for dict in data_dict:
    #print (dict["text"], end = '\n')
    dict["text"] = re.sub(r'[\W_]+',' ', dict["text"])
    print(dict["text"])

# Création et écriture dans le fichier zone d'atterissage
with open("zone_d'atterrissage.json", 'w') as f2:
    json.dump(data_dict, f2)



Goumin des éléphants joueurs la même fatigue même twitter225
 ericbailly24 maxigr04del mes tontons vous avez fait votre part JO prochain on ira en demi final au moins BRAVO à vous SupportriceMazo domie CIV
Ah oui le sommeil là sera compliqué CIV est éliminé des JO Ahi on peut faire ça 
31 juillet journée internationale de la femme africaine jifa
Le pedigree https t co D3Rv7A2BOF
 isabelle170516 leonna julie Steiner2502 Vous avez tt à fait raison le silence incompréhensible du gouver noument et des merdias sur ce très important et dramatique sujet prouve de manière irréfutable leur implication à ce plan diabolique maquillé 
 LynLyna12 leonna julie La grande muette continue et continuera de le rester À part quelques irréductibles à la retraite 
Under wsh 
Les bains d apollon versailles nocturne appollon à Château de Versailles https t co SFGXP06r61
 leonna julie Le rdv aujourd hui aura tenu ses promesses Pour les problèmes de sommeil 1 cachet à la mélatonine 1 80 mg et je dors comme un b

In [ ]:
##c'est la même chose que ce que j'ai fait juste en haut mais j'ai essayé de faire une 
# copie du dictionnaire pour pouvoir faire des modifs sans que le dictionnaire 
# initial ne soit modifier pour pouvoir le réutiliser après, mais ça ne marche pas

#Ouverture du fichier jason
with open('versailles_tweets_100.json') as json_data:
    data_dict = json.load(json_data)

#Effacer tous les caractères spéciaux dans le texte
copy_data = data_dict.copy()
print(copy_data)
for dict in copy_data:
    #print (dict["text"], end = '\n')
    copy_data["text"] = re.sub(r'[\W_]+',' ', copy_data["text"])


# Création et écriture dans le fichier zone d'atterissage
with open("zone_d'atterrissage.json", 'w') as f2:
    json.dump(copy_data, f2)

Identification de l'auteur de la publication

In [7]:
with open('versailles_tweets_100.json') as json_data:
    data = json.load(json_data)

liste_auteurs= []
for dict in data:
    liste_auteurs.append(dict['_id'])
print(liste_auteurs)

['1421616335700824064', '1421599703116943360', '1421599163561742339', '1421591889095057416', '1421582795294617605', '1421581383454052359', '1421575939700445184', '1421569996858269697', '1421563798503272448', '1421562928176803848', '1421560592775761923', '1421558089761935365', '1421551384512585729', '1421550148061212673', '1421547651171078146', '1421499497620836356', '1421499329974579200', '1421487092023050244', '1421472953980571651', '1421429486432686081']


Extraction de la liste des hashtags et la liste des mentions de chaque publication

In [31]:
hash = []
mention = []

for dict in data:
    #Extraction des hashtags
    hashtags = re.findall(r'\B#\w*[a-zA-Z]+\w*', dict['text'])
    hash.append(hashtags)
    #Extraction des mentions
    mentions = re.findall(r'\B@\w*[a-zA-Z]+\w*', dict['text'])
    mention.append(mentions)

liste_hashtags = []
for i in range (len(hash)):
    for e in hash[i]:
        liste_hashtags.append(e)
liste_mentions = []

for i in range (len(mention)):
    for e in mention[i]:
        liste_mentions.append(e)


print(liste_hashtags)
print('\n')
print(liste_mentions)

['#twitter225', '#SupportriceMazo', '#domie', '#CIV', '#CIV', '#jifa', '#versailles', '#nocturne', '#appollon']


['@ericbailly24', '@maxigr04del', '@isabelle170516', '@leonna_julie', '@Steiner2502', '@LynLyna12', '@leonna_julie', '@leonna_julie', '@miliemelo82', '@kilianbridoux', '@LeMeneec', '@Polo82810715', '@lrestistant73', '@Pauluskupa', '@anniemacmanus', '@yebbasmith', '@anniemacmanus', '@AzmiAnees3']


Analyse de sentiments de la publication

In [9]:
liste_sentiments = []
with open("zone_d'atterrissage.json") as f:
   clean_data  = json.load(f)

for dico in clean_data:
    sentiment = TextBlob(dico['text']).sentiment 
    liste_sentiments.append(sentiment[1])
print(liste_sentiments)


[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


Identification du/des topics de la publication 

Top K hashtags

In [ ]:
# création d'un dictionnaire contenant un hashtag et le nombre de fois qu'il apparaît
def K_hashtags(liste_hashtags):
    occurences = []
    dic = {}
    for i in range (len(liste_hashtags)):
            dic[liste_hashtags[i]] = liste_hashtags.count(liste_hashtags[i]) 
    return dic

K_hashtags(liste_hashtags)


Top K utilisateurs mentionnés

In [ ]:
# création d'un dictionnaire contenant un utilisateur et le nombre de fois qu'il est mentionné

def K_mention(liste_mentions):
    occurences = []
    dic = {}
    for i in range (len(liste_mentions)):
            dic[liste_mentions[i]] = liste_mentions.count(liste_mentions[i]) 
    return dic

K_mention(liste_mentions)


Top K utilisateur

In [ ]:
# création d'un dictionnaire contenant un utilisateur et le nombre de fois qu'il a posté un tweet
def K_utilisateur(liste_auteurs):
    occurences = []
    dic = {}
    for i in range (len(liste_auteurs)):
            dic[liste_auteurs[i]] = liste_auteurs.count(liste_auteurs[i]) 
    return dic

K_utilisateur(liste_auteurs)